In [1]:
import numpy as np
import sympy as sp
import math
from sympy import symbols, integrate, pprint, solve
from numpy import pi, sqrt, rad2deg
import numpy as np
import sympy as sp

**!!! All the unit should be in [N] and [mm]**

In [2]:
def check_safe(d, factors, Se, Sut, Sy, M_a, M_m, T_a, T_m):
    """
    d: diameter
    factors: {'K_f': K_f, 'K_fs': K_fs, 'ka': ka, 'kb': kb, 'kc': kc, 'kd':kd, 'ke': ke}
    Se: endurance limit
    Sut, Sy: ultimate and yeilding strength of the material
    return: {'sigma_a_prime': sigma_a_prime, 'sigma_m_prime': sigma_m_prime, 
            'fatigue safety factor': n_f, 'yeilding safety factor': n_y}
    """
    K_f = factors['K_f']
    K_fs = factors['K_fs']
    ka = factors['ka']
    kb = factors['kb']
    kc = factors['kc']
    kd = factors['kd']
    ke = factors['ke']
    
    sigma_a = (32*K_f*M_a)/(np.pi*d**3) 
    tau_a = (16*K_fs*T_a)/(np.pi*d**3)
    sigma_m = (32*K_f*M_m)/(np.pi*d**3)
    tau_m = (16*K_fs*T_m)/(np.pi*d**3)

    sigma_a_prime = (sigma_a**2 + 3*tau_a**2)**(1/2)
    sigma_m_prime = (sigma_m**2 + 3*tau_m**2)**(1/2)
    
    n_f = ((sigma_a_prime/Se) + (sigma_m_prime/Sut))**(-1)
    n_y = Sy/(sigma_a_prime + sigma_m_prime)
    
    return {'sigma_a_prime': sigma_a_prime, 'sigma_m_prime': sigma_m_prime, 
            'fatigue safety factor': n_f, 'yeilding safety factor': n_y}

In [3]:
# unit:[mm]
AD = 30  # D is the mid of the wide of gear
DC = 15   # C is the mid of the wide of scissor
CB = 15 
BO = 30
AC = 45

### Froce Analysis in the xy plane

In [4]:
#数据可以手算，但不要用计算机算出来约分小数点，会有误差

T = 25000 #Nmm  # From the motor
F = 50 #N   #From the scissor
F_Ax = -12.5 #N
F_Bx = -37.5 #N
V_A = F_Ax

In [5]:
# Moment functions
V_AC = V_A
M_A = V_A*(AD + DC)

V_B = F + F_Ax
V_CB = V_B

M_B = V_AC*CB - F_Ax*(AD+DC)

x = sp.symbols('x')
Mfunc_OA = (M_A/AC)*(x)
Mfunc_AB = ((M_B - M_A)/CB)*(x - AC) + M_A #Nmm

In [6]:
V_B #N

37.5

In [7]:
np.array([M_A, M_B])*1e-3  #Nm

array([-0.5625,  0.375 ])

### Start with point D, where the bending moment is high and there is the keyway.

In [8]:
M_a = 12760.61
M_m = 0
T_a = 0
T_m = 25000
M_a, M_m, T_a, T_m  # Nmm

(12760.61, 0, 0, 25000)

Endurance limit (**Should be modified manually when the data change.**)

In [9]:
#Choose 1050 CD steel
Sut = 690 #MPa
Sy = 580 #MPa
ka = 3.02*Sut**(-0.217)


In [10]:
Se_prime = 0.5*Sut  # [MPa]
# Se = ka*kb*kc*kd*ke*Se_prime # [MPa]
Se_prime  # [MPa]

345.0

Based on the calculation above,Based on the inner radius of the gear (12 mm), let’s design the diameter d to be 12 mm. And calculate all the factors. 
And typical keyway standard $r/d$ ratio for support at a keyway: $r/d = 0.02$ from table7-1  
K_t = 2.14 K_ts=3

In [11]:
d = 12 # [mm] Choose the standard value
r = 0.02*d  #  keyway will be the standard r/d=0.02,
r

0.24

In [12]:
# From Fig.(A-15-9&8) and Fig.(6-26&27)
K_t = 2.14
K_ts = 3
q = 0.52
q_s = 0.5

In [13]:
K_f = 1+q*(K_t - 1)
K_fs = 1+q_s*(K_ts - 1)
kb = 1.24*(d)**(-0.107)
kc=kd=ke=1

factors = {'K_f': K_f, 'K_fs': K_fs, 'ka': ka, 'kb': kb, 'kc': kc, 'kd':kd, 'ke': ke}
Se = ka*kb*kc*kd*ke*Se_prime
Se

239.7425261660359

In [14]:
factors

{'K_f': 1.5928,
 'K_fs': 2.0,
 'ka': 0.7310998423692509,
 'kb': 0.9504938076488614,
 'kc': 1,
 'kd': 1,
 'ke': 1}

In [15]:
check_safe(d, factors, Se, Sut, Sy, M_a, M_m, T_a, T_m)

{'sigma_a_prime': 119.80889153512388,
 'sigma_m_prime': 255.24485899157042,
 'fatigue safety factor': 1.149874762902595,
 'yeilding safety factor': 1.5464450073769325}

###  At point E, we will look up data for a specific retaining ring to obtain Kf and Kfs more accurately.
With a quick online search of a retaining ring specification appropriate groove specifications for a retaining ring for a shaft diameter of 11.5mm are obtained as follows: width, a=1mm; depth, 0.25mm; and corner radius at the bottom of grove, r=0.13mm 

In [16]:
M_a = 8507.074
M_m = 0
T_a = 0
T_m = 25000
M_a, M_m, T_a, T_m  # Nmm

(8507.074, 0, 0, 25000)

In [17]:
# From Fig.(A-15-16&17) and Fig.(6-26&27)
K_t = 2.9
K_ts = 1.3
q = 0.51
q_s = 0.55

In [18]:
K_f = 1+q*(K_t - 1)
K_fs = 1+q_s*(K_ts - 1)
ka = ka = 3.02*Sut**(-0.217)
kb = 1.24*(d)**(-0.107)
kc=kd=ke=1
factors = {'K_f': K_f, 'K_fs': K_fs, 'ka': ka, 'kb': kb, 'kc': kc, 'kd':kd, 'ke': ke}
Se = ka*kb*kc*kd*ke*Se_prime
Se

239.7425261660359

In [19]:
factors

{'K_f': 1.9689999999999999,
 'K_fs': 1.165,
 'ka': 0.7310998423692509,
 'kb': 0.9504938076488614,
 'kc': 1,
 'kd': 1,
 'ke': 1}

In [20]:
check_safe(d, factors, Se, Sut, Sy, M_a, M_m, T_a, T_m)

{'sigma_a_prime': 98.73753805438075,
 'sigma_m_prime': 148.68013036258975,
 'fatigue safety factor': 1.5940657591496743,
 'yeilding safety factor': 2.344214152978484}

### Let's consider the force on C

In [21]:
M_a = 6402.31
M_m = 0
T_a = 0
T_m = 25000
M_a, M_m, T_a, T_m  # Nmm

(6402.31, 0, 0, 25000)

In [22]:
r = 1.2 #mm
D = 18 #mm
d = 12 #mm
D/d

1.5

In [23]:
# From Fig.(A-15-9&8) and Fig.(6-26&27)
K_t = 1.7
K_ts = 1.4
q = 0.76
q_s = 0.8

In [24]:
K_f = 1+q*(K_t - 1)
K_fs = 1+q_s*(K_ts - 1)
ka = 3.02*Sut**(-0.217)
kb = 1.24*(d)**(-0.107)

factors = {'K_f': K_f, 'K_fs': K_fs, 'ka': ka, 'kb': kb, 'kc': kc, 'kd':kd, 'ke': ke}
Se = ka*kb*kc*kd*ke*Se_prime
Se

239.7425261660359

In [25]:
factors

{'K_f': 1.532,
 'K_fs': 1.3199999999999998,
 'ka': 0.7310998423692509,
 'kb': 0.9504938076488614,
 'kc': 1,
 'kd': 1,
 'ke': 1}

In [26]:
check_safe(d, factors, Se, Sut, Sy, M_a, M_m, T_a, T_m)

{'sigma_a_prime': 57.816504542171195,
 'sigma_m_prime': 168.46160693443645,
 'fatigue safety factor': 2.0605467858973983,
 'yeilding safety factor': 2.5632174328092696}

### Deflection in the xy plane

In [27]:
def deflection_slope(Mz1, Mz2, E, d1, d2, AC, AB):
    """
    Mz1, Mz2, Mz3:  Nmm
    AE, AH, AB:  mm
    E: MPa
    d1:  mm
    d2:  mm
    return: deflection and slope for each segment
    """
    I1 = pi*d1**4/64
    I2 = pi*d2**4/64
    C1, C2, C3, C4, C5, C6 = symbols('C1, C2, C3, C4, C5, C6',real=True)
    EIdv1dx = integrate(Mz1,x) + C1
    EIv1 = integrate(EIdv1dx,x) + C2

    EIdv2dx = integrate(Mz2,x) + C3
    EIv2 = integrate(EIdv2dx,x) + C4

    eq1 = EIv1.subs(x,0)/E/I1
    eq2 = EIdv1dx.subs(x,AC)/E/I1 - EIdv2dx.subs(x,AC)/E/I2
    eq3 = EIv1.subs(x,AC)/E/I1 
    eq4 = EIv2.subs(x,AB)/E/I2
    
    c1t6 = solve((eq1,eq2,eq3,eq4))
    
    v1 = EIv1.subs(c1t6)/E/I1
    v2 = EIv2.subs(c1t6)/E/I2
    
    dv1dx = EIdv1dx.subs(c1t6)/E/I1
    dv2dx = EIdv2dx.subs(c1t6)/E/I2
    
    defl = {"v1": v1, "v2": v2}
    slop = {"s1": dv1dx, "s2": dv2dx}
    
    return defl, slop

In [28]:
AC= 45
AD= 30
AB = 60
DB = 30
x = sp.symbols('x')
Mfunc_AC = (-12.5)*(x)
Mfunc_CB = (37.5)*(x) -2250 #Nmm

In [29]:
d1 = 12 #mm
d2 = 12 #mm
E = 205.0e3 # N/mm/mm  typical value for steel
Mz1 = Mfunc_AC
Mz2 = Mfunc_CB


In [30]:
defl, slop = deflection_slope(Mz1, Mz2, E, d1, d2, AC, AB)

In [31]:
v1, v2 = defl['v1'], defl['v2']
s1, s2 = slop['s1'], slop['s2']
print('v1:', v1)
print('v2:', v2)

v1: -9.98412520650754e-9*x**3 + 2.02178535431777e-5*x
v2: 2.99523756195226e-8*x**3 - 5.39142761151407e-6*x**2 + 0.000262832096061311*x - 0.00283049949604483


In [32]:
print('s1:', s1)
print('s2:', s2)

s1: 2.02178535431777e-5 - 2.99523756195226e-8*x**2
s2: 8.98571268585678e-8*x**2 - 1.07828552230281e-5*x + 0.000262832096061311


### Checking deflections and slopes

#### At gear D:

In [33]:
rotzD = float(s1.subs(x,AD))
rotyD = 0

slD1 = sqrt(rotzD*rotzD + rotyD*rotyD)
slD1 #rad

6.739284514392614e-06

In [55]:
rotD = float(v1.subs(x,AD))
rotD = 0
slDd = sqrt(rotD*rotD + rotD*rotD)
slDd, rotD #rad

(0.0, 0)

#### At scissor C:

In [34]:
rotzC = float(s1.subs(x,AC))
rotyC = 0

slC1 = sqrt(rotzC*rotzC + rotyC*rotyC)
slC1 #rad

4.043570708635555e-05

#### At bearing B:

In [35]:
rotzB = float(s2.subs(x,AB))
rotyB = 0

slB1 = sqrt(rotzB*rotzB + rotyB*rotyB)
slB1 #rad

6.0653560629533496e-05

### Deflection in the xz plane

In [36]:
def deflection_slope(Mz1, Mz2, E, d1, d2, AD, AB):
    """
    Mz1, Mz2, Mz3:  Nmm
    AE, AH, AB:  mm
    E: MPa
    d1:  mm
    d2:  mm
    return: deflection and slope for each segment
    """
    I1 = pi*d1**4/64
    I2 = pi*d2**4/64
    C1, C2, C3, C4, C5, C6 = symbols('C1, C2, C3, C4, C5, C6',real=True)
    EIdv1dx = integrate(Mz1,x) + C1
    EIv1 = integrate(EIdv1dx,x) + C2

    EIdv2dx = integrate(Mz2,x) + C3
    EIv2 = integrate(EIdv2dx,x) + C4

    eq1 = EIv1.subs(x,0)/E/I1
    eq2 = EIdv1dx.subs(x,AD)/E/I1 - EIdv2dx.subs(x,AD)/E/I2
    eq3 = EIv1.subs(x,AD)/E/I1 
    eq4 = EIv2.subs(x,AB)/E/I2
    
    c1t6 = solve((eq1,eq2,eq3,eq4))
    
    v1 = EIv1.subs(c1t6)/E/I1
    v2 = EIv2.subs(c1t6)/E/I2
    
    dv1dx = EIdv1dx.subs(c1t6)/E/I1
    dv2dx = EIdv2dx.subs(c1t6)/E/I2
    
    defl = {"v1": v1, "v2": v2}
    slop = {"s1": dv1dx, "s2": dv2dx}
    
    return defl, slop

In [37]:
AC=45
AD=30
AB = 60
DC = 15
DB = 30
x = sp.symbols('x')
Mfunc_AD = (425.17)*(x)
Mfunc_DB = (-425.17)*(x) +25510.2 #Nmm

In [38]:
d1 = 12 #mm
d2 = 12 #mm
E = 205.0e3 # N/mm/mm  typical value for steel
Mz1 = Mfunc_AD
Mz2 = Mfunc_DB


In [39]:
defl, slop = deflection_slope(Mz1, Mz2, E, d1, d2, AD, AB)
AB

60

In [40]:
v1, v2 = defl['v1'], defl['v2']
s1, s2 = slop['s1'], slop['s2']
print('v1:', v1)
print('v2:', v2)

v1: 3.39596041124065e-7*x**3 - 0.000305636437011657*x
v2: -3.39596041124065e-7*x**3 + 6.11272874023317e-5*x**2 - 0.00213945505908161*x - 0.0183381862206996


In [41]:
print('s1:', s1)
print('s2:', s2)

s1: 1.01878812337219e-6*x**2 - 0.000305636437011657
s2: -1.01878812337219e-6*x**2 + 0.000122254574804663*x - 0.00213945505908161


### Checking deflections and slopes

#### At gear D:

In [49]:
rotzD = float(s1.subs(x,AD))
rotyD = 0

slD2 = sqrt(rotzD*rotzD + rotyD*rotyD)
slD2 #rad

0.0006112728740233174

#### At scissor C:

In [52]:
rotzC = float(s1.subs(x,AC))
rotyC = 0

slC2 = sqrt(rotzC*rotzC + rotyC*rotyC)
slC2 #rad

0.0017574095128170362

#### At bearing B:

In [44]:
rotzB = float(s2.subs(x,AB))
rotyB = 0

slB2 = sqrt(rotzB*rotzB + rotyB*rotyB)
slB2 #rad

0.0015281821850582924

In [45]:
slD = sqrt(slD1*slD1 + slD2*slD2)
slD

0.0006113100232063044

In [46]:
slC = sqrt(slC1*slC1 + slC2*slC2)
slC

0.0017578746377792092

In [47]:
slB = sqrt(slB1*slB1 + slB2*slB2)
slB

0.0015293853815002212